# Step 1: Company Setup - Building the Organization

## 🏢 From "I" to "We"

In Step 0, you created **your identity**. You exist in the system.

But you don't work alone. You work for a **company**. You have **colleagues**. You use **systems** and **equipment**.

This notebook creates the **organizational context** that surrounds you.

## 🎯 The Challenge: Modeling an Organization

A company isn't just a name. It's a complex web of:
- **People**: Employees with accounts, emails, and roles
- **Systems**: Servers, applications, databases
- **Assets**: Laptops, phones, network equipment
- **Relationships**: Who works where? Who uses what? Who manages which systems?

All of these are **identities** in the STIX sense - entities that can be actors, targets, or witnesses in security incidents.

## 🔁 The Pattern Returns: Pattern 3.2 at Scale

Remember Pattern 3.2 from Step 0?
1. Create user-account (Level 0)
2. Create email-addr (Level 0)
3. Create identity (Level 2)

**You'll use it again. Many times.**

But now with variations:
- **Company identity**: No account or email (it's an organization)
- **Employee identities**: Same 3-step pattern as Step 0
- **System identities**: Systems don't have emails (just accounts)
- **Asset identities**: Assets might not have either (just identity)

**Same pattern, different parameters.** This is the power of Pattern 3.2.

## 🗺️ The Journey: Four Acts

**Act 1: Setup** (Libraries, paths, utilities)
- Same foundation as Step 0

**Act 2: Create Company Identity** (The organization itself)
- Initialize company context directory
- Create company identity object
- Establish multi-tenant storage

**Act 3: Populate the Organization** (People, systems, assets)
- Create 3 employee identities (Pattern 3.2 × 3)
- Create 1 IT system identity (Exchange server)
- Create 3 hardware asset identities (Laptops)

**Act 4: Summary** (What we built, why it matters)
- Inventory of objects created
- Storage structure explained
- Next steps outlined

## ? What You'll Create

**Total**: ~23 STIX objects
- 1 company identity (organization)
- 9 employee objects (3 people × 3 objects)
- 1 IT system identity
- 3 hardware asset identities
- Relationships (company → employees, company → systems, company → assets)

All stored in: `context_mem/identity--{company-uuid}/`

## 🎓 What You'll Learn

1. **Pattern 3.2 Reuse**: Same pattern, different contexts
2. **Multi-Tenant Storage**: Company context separate from user context
3. **Category-Based Organization**: users.json, systems.json, assets.json
4. **Organizational Modeling**: How to represent complex organizations in STIX
5. **Context Routing**: How context_map.json routes to company contexts

## 📋 Prerequisites

✅ **Step_0_User_Setup.ipynb** must be complete
- You need YOUR identity to create the company
- Company context builds on user context

Let's begin! 🚀

## Act 1: Setup - Scene 1: Import STIX Libraries

Same libraries as Step 0 - we're creating the same types of objects (identities, accounts, emails).

**Plus**: We'll use the **IdentityContact extension** for richer organizational data (phone numbers, social media, etc.).

In [28]:
import sys
!{sys.executable} -m pip install stixorm

from stixorm.module.authorise import import_type_factory
from stixorm.module.definitions.stix21 import (
    Identity, EmailAddress, UserAccount, Relationship, Bundle
)
from stixorm.module.definitions.os_threat import (
    IdentityContact, EmailContact, SocialMediaContact, ContactNumber
)
from stixorm.module.typedb_lib.instructions import ResultStatus, Result
from stixorm.module.parsing import parse_objects

import_type = import_type_factory.get_all_imports()

import logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

print("✅ STIX 2.1 libraries loaded")
print("✅ OS Threat extensions loaded (IdentityContact for enhanced company data)")
print("✅ Act 1, Scene 1 complete")

✅ OS Threat extensions loaded (IdentityContact for enhanced company data)
✅ Act 1, Scene 1 complete



[notice] A new release of pip is available: 24.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## Act 1: Scene 2: Configure Python Path

Set up paths so we can use Brett Blocks utilities.

In [29]:
import sys
sys.path.append('../')
import os

cwd = os.getcwd()

print(f"✅ Working directory: {cwd}")
print("✅ Act 1, Scene 2 complete")

✅ Working directory: c:\projects\brett_blocks\Orchestration
✅ Act 1, Scene 2 complete


## Act 1: Scene 3: Import Company Context Utilities

**New utilities** for company-specific operations:
- `invoke_create_company_context()`: Creates the company context directory
- `invoke_save_company_context_block()`: Saves objects to company context (with categories)
- Same object creation utilities: `invoke_make_identity_block()`, etc.

We'll also load the data configurations for:
- 1 company
- 3 employees
- 1 IT system
- 3 hardware assets

In [30]:
import json

# Company-specific utilities
from Utilities.local_make_general import invoke_create_company_context
from Utilities.local_make_general import invoke_save_company_context_block
from Utilities.local_make_general import invoke_update_company_relations_block

# Standard utilities (same as Step 0)
from Utilities.util import emulate_ports, unwind_ports, conv
from Utilities.local_make_sdo import invoke_make_identity_block
from Utilities.local_make_sco import invoke_make_user_account_block, invoke_make_email_addr_block
from Utilities.local_make_sro import invoke_sro_block

# Paths
path_base = "../Block_Families/StixORM/"
results_base = "../Orchestration/Results/"

# Company identity configuration
company_base = {
    "ident": "SDO/Identity/identity_TR_user_company.json",
    "results": "step0/company"
}

# Employee configurations (3 employees)
company_users_data = [
    {
        "who": "usr1",
        "email": "SCO/EmailAddress/email_addr_IT_user1.json",
        "acct": "SCO/UserAccount/usr_account_IT_user1.json",
        "ident": "SDO/Identity/identity_IT_user1.json",
        "results": "step0/harry1"
    },
    {
        "who": "usr2",
        "email": "SCO/EmailAddress/email_addr_IT_user2.json",
        "acct": "SCO/UserAccount/usr_account_IT_user2.json",
        "ident": "SDO/Identity/identity_IT_user2.json",
        "results": "step0/jane2"
    },
    {
        "who": "usr3",
        "email": "SCO/EmailAddress/email_addr_IT_user3.json",
        "acct": "SCO/UserAccount/usr_account_IT_user3.json",
        "ident": "SDO/Identity/identity_IT_user3.json",
        "results": "step0/sally3"
    }
]

# IT systems (1 Exchange server)
systems_base = [
    {
        "data_path": "SDO/Identity/identity_Exchange.json",
        "results": "step0/system1"
    }
]

# Hardware assets (3 laptops)
assets_base = [
    {
        "data_path": "SDO/Identity/identity_Laptop1.json",
        "results": "step0/asset1"
    },
    {
        "data_path": "SDO/Identity/identity_Laptop2.json",
        "results": "step0/asset2"
    },
    {
        "data_path": "SDO/Identity/identity_Laptop3.json",
        "results": "step0/asset3"
    }
]

print("✅ Company context utilities loaded")
print(f"✅ Configuration loaded:")
print(f"   - 1 company identity")
print(f"   - {len(company_users_data)} employees")
print(f"   - {len(systems_base)} IT systems")
print(f"   - {len(assets_base)} hardware assets")
print("✅ Act 1 complete - Ready to create company context!")


✅ Company context utilities loaded
✅ Configuration loaded:
   - 1 company identity
   - 3 employees
   - 1 IT systems
   - 3 hardware assets
✅ Act 1 complete - Ready to create company context!


## Act 2: Create Company Identity - The Organization Itself

**This is different from Step 0.** You're not creating a person - you're creating an **organization**.

### What Happens Here

1. **Create Company Identity Object**: The organization as a STIX Identity
   - No user-account (companies don't log in)
   - No email-addr (organizations don't have individual emails)
   - Just the identity itself (name, sector, description)

2. **Initialize Company Context Directory**: `/identity--{company-uuid}/`
   - This becomes the "home" for all company objects
   - Separate from your personal context (`/usr/`)
   - Enables **multi-tenant storage** (multiple companies in one system)

3. **Update Context Routing**: `context_map.json`
   - Maps company UUID to its context directory
   - Enables fast lookups: "Where does Company X store its data?"

### Why This Matters

Later, when you create an incident, you'll need to specify:
- **Who was affected?** → Company identity (the victim)
- **Who investigated?** → Your identity (the analyst)
- **Who reported?** → Employee identity (the witness)

The company identity is the **root of the organizational tree**.

In [31]:
# Create company identity object
comp_ident = invoke_make_identity_block(
    company_base["ident"], 
    company_base["results"]
)

# Define output paths
ident_results_obj_path = results_base + company_base["results"] + "__ident.json"
ident_results_context_path = results_base + "step0/context/company__ident_context.json"

# Initialize company context and save identity
result = invoke_create_company_context(ident_results_obj_path, ident_results_context_path)

print(f"✅ Company identity created: {comp_ident['type']} - {comp_ident['id'][:40]}...")
print(f"✅ Company context initialized: /identity--{comp_ident['id'][9:45]}/")
print(f"✅ Context routing updated: context_map.json")
print(f"✅ Result: {result}")
print("")
print("🏢 The organization now exists in the system!")
print("✅ Act 2 complete")

type identity->{'identity_form': {'base_required': {'type': 'identity', 'spec_version': '2.1', 'id': '', 'created': '', 'modified': ''}, 'base_optional': {'created_by_ref': '', 'revoked': None, 'labels': [], 'lang': '', 'external_references': [], 'object_marking_refs': [], 'granular_markings': [], 'defanged': None}, 'object': {'name': 'Example Company', 'description': 'An Example Organisation', 'identity_class': 'organization', 'sectors': 'technology'}, 'extensions': {}, 'sub': {}}}
type identity-><class 'dict'>, type email-><class 'NoneType'>, type user acct-><class 'NoneType'>
Step 1 >>
Step 2 >>
Step 3 >>
{
    "type": "identity",
    "spec_version": "2.1",
    "id": "identity--854a7d50-b584-4826-896f-41c20887fc45",
    "created": "2025-11-17T02:25:26.890Z",
    "modified": "2025-11-17T02:25:26.890Z",
    "name": "Example Company",
    "description": "An Example Organisation",
    "identity_class": "organization",
    "sectors": [
        "technology"
    ]
}
%%%% 
input->{'company'

## Act 3: Populate the Organization - Scene 1: Create Employees

**Pattern 3.2 returns!** Same 3 steps as Step 0, repeated for each employee.

For each of the 3 employees:
1. Create user-account (Level 0)
2. Create email-addr (Level 0)
3. Create identity (Level 2)

### Storage Difference: Category-Based Organization

**Step 0**: Stored in `/usr/cache_me.json` or `/usr/cache_team.json`
**Step 1**: Stored in `/identity--{company-uuid}/users.json`

Why the difference?
- **User context** (`/usr/`): Personal data (you and your team)
- **Company context** (`/identity--{uuid}/`): Organizational data (company employees)

Within company context, objects are organized by **category**:
- `users.json` - Employee identities
- `systems.json` - IT system identities
- `assets.json` - Hardware asset identities
- `company.json` - Company identity
- `edges.json` - Relationships

This enables:
- **Fast filtering**: "Show me all employees"
- **Role-based access**: Different permissions per category
- **Clean separation**: People vs systems vs assets

In [32]:
# Process each employee (Pattern 3.2 × 3)
context_type = {"context_type": "users"}  # Store in users.json

for ident in company_users_data:
    if ident["who"][:3] != "usr": continue
    
    print(f"\n👤 Creating employee: {ident['who']}")
    
    # Step 1: Create user-account (Level 0)
    user_acct = invoke_make_user_account_block(ident["acct"], ident["results"])
    acct_obj_path = results_base + ident["results"] + "__usr_acct.json"
    acct_context_path = results_base + f"step0/context/usr_{ident['results'][-1]}__usr_acct_context.json"
    invoke_save_company_context_block(acct_obj_path, acct_context_path, context_type)
    print(f"   ✅ User account created (Level 0)")
    
    # Step 2: Create email-addr (Level 0)
    user_email = invoke_make_email_addr_block(ident["email"], ident["results"], user_acct)
    email_obj_path = results_base + ident["results"] + "__email.json"
    email_context_path = results_base + f"step0/context/usr_{ident['results'][-1]}__email.json"
    invoke_save_company_context_block(email_obj_path, email_context_path, context_type)
    print(f"   ✅ Email address created (Level 0)")
    
    # Step 3: Create identity (Level 2)
    user_ident = invoke_make_identity_block(
        ident["ident"], ident["results"], 
        email_results=user_email, acct_results=user_acct
    )
    ident_obj_path = results_base + ident["results"] + "__ident.json"
    ident_context_path = results_base + f"step0/context/usr_{ident['results'][-1]}__ident_context.json"
    invoke_save_company_context_block(ident_obj_path, ident_context_path, context_type)
    print(f"   ✅ Identity created (Level 2)")
    print(f"   ✅ Stored in: users.json")

print(f"\n🎉 Pattern 3.2 used 3 times - 3 employees created")
print(f"✅ Total objects: 9 (3 employees × 3 objects)")
print(f"✅ Storage: /identity--{{company-uuid}}/users.json")
print("✅ Act 3, Scene 1 complete")


👤 Creating employee: usr1
{
    "type": "user-account",
    "spec_version": "2.1",
    "id": "user-account--597ad4d4-35ba-585d-8f6d-134a75032f9b",
    "user_id": "79563902",
    "account_login": "nsmith",
    "account_type": "sales,",
    "display_name": "Naive Smith"
}
input data->{'stix_object': {'type': 'user-account', 'spec_version': '2.1', 'id': 'user-account--597ad4d4-35ba-585d-8f6d-134a75032f9b', 'user_id': '79563902', 'account_login': 'nsmith', 'account_type': 'sales,', 'display_name': 'Naive Smith'}, 'context_type': {'context_type': 'users'}}
from ports 
stix_object->{'type': 'user-account', 'spec_version': '2.1', 'id': 'user-account--597ad4d4-35ba-585d-8f6d-134a75032f9b', 'user_id': '79563902', 'account_login': 'nsmith', 'account_type': 'sales,', 'display_name': 'Naive Smith'}
context type->users
   ✅ User account created (Level 0)
   ✅ User account created (Level 0)
{
    "type": "email-addr",
    "spec_version": "2.1",
    "id": "email-addr--4722424c-7012-56b0-84d5-01d076f

## Act 3: Scene 2: Create IT Systems

**IT systems** are also identities, but with a twist:
- They have accounts (system accounts, service principals)
- They typically **don't** have email addresses
- They're represented as Identity objects with `identity_class: "system"`

### Example: Exchange Server

In this notebook, we create an Exchange server identity. Why?

Later, in incident investigation, you might need to say:
- "The Exchange server was compromised" → The system is the **target**
- "The attacker accessed Exchange logs" → The system is the **evidence source**
- "Exchange sent phishing emails" → The system is the **attack vector**

Systems are **actors in the security narrative**, just like people.

In [33]:
# Process IT systems (just identity objects, no Pattern 3.2)
context_type = {"context_type": "systems"}  # Store in systems.json

for IT_system in systems_base:
    print(f"\n🖥️ Creating IT system: {IT_system['results']}")
    
    # Create system identity (no account/email needed for this example)
    system_ident = invoke_make_identity_block(
        IT_system["data_path"], 
        IT_system["results"]
    )
    
    # Save to company context
    ident_obj_path = results_base + IT_system["results"] + "__ident.json"
    ident_context_path = results_base + f"step0/context/system{IT_system['results'][-1]}__ident_context.json"
    result = invoke_save_company_context_block(ident_obj_path, ident_context_path, context_type)
    
    print(f"   ✅ System identity created: {system_ident['type']} - {system_ident['id'][:40]}...")
    print(f"   ✅ Stored in: systems.json")
    print(f"   ✅ Result: {result}")

print(f"\n🎉 IT systems created: {len(systems_base)}")
print("✅ Act 3, Scene 2 complete")


🖥️ Creating IT system: step0/system1
type identity->{'identity_form': {'base_required': {'type': 'identity', 'spec_version': '2.1', 'id': '', 'created': '', 'modified': ''}, 'base_optional': {'created_by_ref': '', 'revoked': None, 'labels': [], 'lang': '', 'external_references': [{'source_name': 'Exchange Server', 'description': 'our email server', 'url': 'https://mail.echange.microsoft.com', 'external_id': 'admin'}], 'object_marking_refs': [], 'granular_markings': [], 'defanged': None}, 'object': {'name': 'Microsoft Exchange', 'description': 'My Local Exchange Server', 'roles': ['server'], 'identity_class': 'system', 'sectors': ['technology']}, 'extensions': {}, 'sub': {}}}
type identity-><class 'dict'>, type email-><class 'NoneType'>, type user acct-><class 'NoneType'>
Step 1 >>
Step 2 >>
Step 3 >>
{
    "type": "identity",
    "spec_version": "2.1",
    "id": "identity--f33b0f4a-273c-4a12-9100-d6d8fa630b40",
    "created": "2025-11-17T02:25:27.671Z",
    "modified": "2025-11-17T02:

## Act 3: Scene 3: Create Hardware Assets

**Hardware assets** are physical devices that can be:
- **Targets**: "Laptop was stolen"
- **Evidence sources**: "Laptop logs show unauthorized access"
- **Attack vectors**: "Laptop was used to launch attack"

Like systems, assets are represented as Identity objects with `identity_class: "asset"` or similar.

### Example: Laptops

We're creating 3 laptop identities. In a real incident, you might need to:
- Track which laptop was compromised
- Identify which employees use which laptops
- Understand the hardware inventory affected by an incident

**Assets complete the organizational picture**: people + systems + assets = complete organization.

In [34]:
# Process hardware assets
context_type = {"context_type": "assets"}  # Store in assets.json

for i, asset in enumerate(assets_base):
    print(f"\n🔧 Creating hardware asset: {asset['results']}")
    
    # Create asset identity
    asset_ident = invoke_make_identity_block(
        asset["data_path"], 
        asset["results"]
    )
    
    # Save to company context
    ident_obj_path = results_base + asset["results"] + "__ident.json"
    ident_context_path = results_base + f"step0/context/hardware{asset['results'][-1]}{i}__ident_context.json"
    result = invoke_save_company_context_block(ident_obj_path, ident_context_path, context_type)
    
    print(f"   ✅ Asset identity created: {asset_ident['type']} - {asset_ident['id'][:40]}...")
    print(f"   ✅ Stored in: assets.json")
    print(f"   ✅ Result: {result}")

print(f"\n🎉 Hardware assets created: {len(assets_base)}")
print("✅ Act 3 complete - Organization fully populated!")


🔧 Creating hardware asset: step0/asset1
type identity->{'identity_form': {'base_required': {'type': 'identity', 'spec_version': '2.1', 'id': '', 'created': '', 'modified': ''}, 'base_optional': {'created_by_ref': '', 'revoked': None, 'labels': [], 'lang': '', 'external_references': [{'source_name': 'HP Ultima 10', 'description': '16GB RAM, 1TB SD, Microsoft Windows 11 Professional', 'url': 'https://mail.echange.microsoft.com', 'external_id': 'Asset--629456a'}], 'object_marking_refs': [], 'granular_markings': [], 'defanged': None}, 'object': {'name': 'HP Laptop 1', 'description': "Usr 1's Laptop", 'roles': ['laptop'], 'identity_class': 'asset', 'sectors': ['technology']}, 'extensions': {}, 'sub': {}}}
type identity-><class 'dict'>, type email-><class 'NoneType'>, type user acct-><class 'NoneType'>
Step 1 >>
Step 2 >>
Step 3 >>
{
    "type": "identity",
    "spec_version": "2.1",
    "id": "identity--53ce7a3f-5308-4692-afeb-40b2e65eda96",
    "created": "2025-11-17T02:25:27.803Z",
    "

## Optional: Create Organizational Relationships

**Relationships** (SROs) connect the entities we just created:
- `employed-by`: Employees → Company
- `part-of`: Systems → Company
- `owns`: Company → Assets
- `uses`: Employees → Assets

These relationships enable sophisticated queries:
- "Show me all employees of Company X"
- "Which systems does Employee Y administer?"
- "What assets are owned by Company Z?"

**Note**: Relationship creation is currently disabled for debugging. Enable when needed.

In [35]:
# Relationship creation (currently disabled)
stix_object_path = "../Block_Families/Context/Update_context/relationship_type.json"
results_path = results_base + "step0/_relations_made.json"

print("🔗 Organizational relationships:")
print("   - employed-by: employees → company")
print("   - part-of: systems → company")
print("   - owns: company → assets")
print("")
print("⚠️ Relationship creation currently disabled for debugging")
print("🔧 Uncomment the following lines when ready:")
print("")
print("   # result = invoke_update_company_relations_block(stix_object_path, results_path)")
print("   # print(f'✅ Relationships created: {result}')")
print("")
print("✅ Company structure is complete without relationships")

🔗 Organizational relationships:
   - employed-by: employees → company
   - part-of: systems → company
   - owns: company → assets

⚠️ Relationship creation currently disabled for debugging
🔧 Uncomment the following lines when ready:

   # result = invoke_update_company_relations_block(stix_object_path, results_path)
   # print(f'✅ Relationships created: {result}')

✅ Company structure is complete without relationships


## Act 4: Summary - The Organization Is Complete

### ✅ Objects Created

**Total**: ~13 STIX objects + relationships

**Company Identity:**
- 1× identity (organization) → `company.json`

**Employee Identities (Pattern 3.2 × 3):**
- 3× user-account (Level 0)
- 3× email-addr (Level 0)
- 3× identity (Level 2)
- All → `users.json`

**IT System Identities:**
- 1× identity (Exchange server) → `systems.json`

**Hardware Asset Identities:**
- 3× identity (Laptops) → `assets.json`

**Relationships (when enabled):**
- employed-by (employees → company)
- part-of (systems → company)
- owns (company → assets)
- All → `edges.json`

### 📁 Context Memory Structure

```
context_mem/
├── context_map.json                    # Routes company UUID → context directory
├── usr/                                # Your personal context (from Step 0)
│   ├── cache_me.json                   # YOUR identity
│   └── cache_team.json                 # Your team members
└── identity--{company-uuid}/           # Company context (NEW!)
    ├── company.json                    # Company identity
    ├── users.json                      # 3 employees (9 objects)
    ├── systems.json                    # 1 IT system
    ├── assets.json                     # 3 hardware assets
    └── edges.json                      # Relationships
```

### 🎓 What You Learned

1. **Pattern 3.2 Reuse**: Same pattern for employees as Step 0 (your identity)
2. **Multi-Tenant Storage**: Company context separate from user context
3. **Category-Based Organization**: users/systems/assets organized separately
4. **Identity Classes**: People, organizations, systems, and assets all as identities
5. **Context Routing**: `context_map.json` enables multi-company support

### 🔍 Pattern Evolution: From Simple to Complex

**Step 0 (Your Identity)**:
- Pattern 3.2 × 1 (just you)
- Storage: `/usr/cache_me.json`
- Single tenant (one user)

**Step 1 (Company)**:
- Pattern 3.2 × 3 (employees)
- Plus: Company, systems, assets
- Storage: `/identity--{uuid}/[category].json`
- Multi-tenant (multiple companies supported)
- Category-based (organized by type)

**Same foundation, more sophisticated organization.**

### 🚀 Next Steps: Create a Security Incident

You're now ready for **Step 2: Create Incident with an Alert**

In Step 2, you'll:
- Create a phishing incident (Pattern 3.1)
- Create an alert about the incident (Pattern 3.3)
- Create an event timeline (Pattern 3.4-3.8)
- Reference the company and employees you just created

**Example incident flow:**
1. Employee receives phishing email → Uses employee identity from Step 1
2. Employee reports to security team → Uses your identity from Step 0
3. Incident created at company → Uses company identity from Step 1
4. Alert triggers for system → Uses Exchange system from Step 1

### 💡 Why This Matters: The Organizational Graph

The objects you created form a **graph**:
- **Nodes**: Identities (company, employees, systems, assets)
- **Edges**: Relationships (employed-by, uses, owns)

This graph enables:
- **Impact Analysis**: "If Exchange is compromised, which employees are affected?"
- **Attribution**: "Which employee's laptop was used in the attack?"
- **Collaboration**: "Who else in the company can help investigate?"
- **Reporting**: "Generate org chart showing affected departments"

**The graph preserves context.** Every incident exists within this organizational structure.

### 🎉 Company Setup Complete!

You now have:
- ✅ YOUR identity (Step 0)
- ✅ COMPANY identity (Step 1)
- ✅ EMPLOYEE identities (Step 1)
- ✅ SYSTEM identities (Step 1)
- ✅ ASSET identities (Step 1)

**The organizational foundation is set. Time to investigate threats.** 🚀

---

**Next**: Open `Step_2_Create_Incident_with_an_Alert.ipynb` to create your first security incident!